In [1]:
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.core import Dense,Dropout
import random
import numpy as np
import pandas as pd
from operator import add

Using TensorFlow backend.


In [5]:
class DQNAgent(object):
  
  def __init__(self):
    #Initializations
    self.reward=0
    self.gamma=0.9
    self.dataframe=pd.DataFrame()
    self.short_memory=np.array([])
    self.agent_target=1
    self.agent_predict=0
    self.learning_rate=0.0005
    self.model=self.network("weights.hdf5") #found on github repo:
    self.epsilon=0
    self.actual=[]
    self.memory=[]
    
  def get_state(self,game,player,food):
    #states for all positions of each item of the game
    
     state = [
         (player.x_change == 20 and player.y_change == 0 and ((list(map(add, player.position[-1], [20, 0])) in player.position) or
         player.position[-1][0] + 20 >= (game.game_width - 20))) or (player.x_change == -20 and player.y_change == 0 and ((list(map(add, player.position[-1], [-20, 0])) in player.position) or
         player.position[-1][0] - 20 < 20)) or (player.x_change == 0 and player.y_change == -20 and ((list(map(add, player.position[-1], [0, -20])) in player.position) or
         player.position[-1][-1] - 20 < 20)) or (player.x_change == 0 and player.y_change == 20 and ((list(map(add, player.position[-1], [0, 20])) in player.position) or
         player.position[-1][-1] + 20 >= (game.game_height-20))),  # danger straight

          (player.x_change == 0 and player.y_change == -20 and ((list(map(add,player.position[-1],[20, 0])) in player.position) or
          player.position[ -1][0] + 20 > (game.game_width-20))) or (player.x_change == 0 and player.y_change == 20 and ((list(map(add,player.position[-1],
          [-20,0])) in player.position) or player.position[-1][0] - 20 < 20)) or (player.x_change == -20 and player.y_change == 0 and ((list(map(
          add,player.position[-1],[0,-20])) in player.position) or player.position[-1][-1] - 20 < 20)) or (player.x_change == 20 and player.y_change == 0 and (
          (list(map(add,player.position[-1],[0,20])) in player.position) or player.position[-1][
          -1] + 20 >= (game.game_height-20))),  # danger right

          (player.x_change == 0 and player.y_change == 20 and ((list(map(add,player.position[-1],[20,0])) in player.position) or
          player.position[-1][0] + 20 > (game.game_width-20))) or (player.x_change == 0 and player.y_change == -20 and ((list(map(
          add, player.position[-1],[-20,0])) in player.position) or player.position[-1][0] - 20 < 20)) or (player.x_change == 20 and player.y_change == 0 and (
          (list(map(add,player.position[-1],[0,-20])) in player.position) or player.position[-1][-1] - 20 < 20)) or (
          player.x_change == -20 and player.y_change == 0 and ((list(map(add,player.position[-1],[0,20])) in player.position) or
          player.position[-1][-1] + 20 >= (game.game_height-20))), #danger left
         
         
          player.x_change==-20 #left
          player.x_change==20 #right
          player.y_change==-20 #up
          player.y_change==20 #down
          food.x_food < player.x #food left
          food.x_food > player.x #food_right
          food.y_food < player.y #food up
          food.y_food > player.y #food down
     ]
      
    for i in range(len(state)):
      if state[i]:
        state[i]=1
      else:
        state[i]=0
      
    return np.asarray(state)
  
  def set_reward(self,player,crash):
    self.reward=0
    if crash:
      #deduct
      self.reward=-10
      return self.reward
    if player.eaten:
      #award
      self.reward=10
    return self.reward
  
  def network(self,weights=None):#weights from github were included in init()
    #DQN
    model=Sequential()
    model.add(Dense(output_dim=120,activation='relu',input_dim=11))
    model.add(Dropout(0.15))
    model.add(Dense(output_dim=120,activation='relu'))
    model.add(Dropout(0.15))
    model.add(Dense(output_dim=120,activation='relu'))
    model.add(Dropout(0.15))
    model.add(Dense(output_dim=3,activation='softmax'))
    model.compile(loss='mse',optimizer=Adam(self.learning_rate)) #Mean Squared Error
    
    if weights:
      model.load_weights(weights)
    return model
  
  def remember(self,state,action,rewardnext_state,done):
    self.memory.append((state,action,reward,next_state,done))
    
  def replay_new(self,memory):
    if len(memory)>1000:
      minibatch=random.sample(memory,1000)
    else:
      minibatch=memory
    
    for state,action,reward,next_state, done in minibatch:
      target=reward
      if not done:
        target=reward+self.gamma*np.amax(self.model.predict(np.array([next_state]))[0])
        
      target_f=self.model.predict(np.array([state]))
      target_f[0][np.argmax(action)]=target
      self.model.fit(np.array([state]),target_f,epochs=1,verbose=0)
      
  def train_short_memory(self,state,action,reward,next_state,done):
    target=reward
    if not done:
      target=reward + self.gamma*np.amax(self.model.predict(next_shape.reshape((1,11)))[0])
    
    target_f=self.model.predict(state.reshape((1,11)))
    target_f[0][np.argmax(action)]=target
    self.model.fit(state.reshape((1,11)),target_f,epochs=1,verbose=0)
    
  
         
    

SyntaxError: ignored